In [ ]:
!pip install afinn
!python -m textblob.download_corpora
!pip install -U textblob
!pip install vaderSentiment
!pip install pyemd
!pip install unidecode
!pip install -U gensim

Defaulting to user installation because normal site-packages is not writeable
/bin/bash: python: command not found
Defaulting to user installation because normal site-packages is not writeable
^C
ERROR: Operation cancelled by user
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [7]:
import re
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
import string
from unidecode import unidecode
import pandas as pd
import bz2
import gensim
import warnings
import numpy as np
from gensim.models import word2vec
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
from tqdm._tqdm_notebook import tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

warnings.filterwarnings('ignore')
tqdm_notebook.pandas()

import matplotlib.pyplot as plt
%matplotlib inline

# Carregando os embeddings

Aqui vamos utilizar os embeddings para realizar as seguintes atividades:

- análise de simlaridade
- classificação de documentos

<b> Carregue os embeddings treinados, como vimos na Aula 2. É o mesmo arquivo que iremos utilizar</b>

Link: https://drive.google.com/open?id=1zI8pGfbUHuU_0wY_FV4tD6w6ZCUJTQbh

In [8]:
%%time
newfilepath = "embedding_wiki_100d_pt.txt"
filepath = "ptwiki_20180420_100d.txt.bz2"
with open(newfilepath, 'wb') as new_file, bz2.BZ2File(filepath, 'rb') as file:
    for data in iter(lambda : file.read(100 * 1024), b''):
        new_file.write(data)

CPU times: user 1min 1s, sys: 1.27 s, total: 1min 2s
Wall time: 1min 3s


In [9]:
# carregar
%%time
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(filepath, binary=False)

# Similaridade de Documentos

Para realizar a similaridade entre documentos, utilize as frases abaixo:

In [10]:
frase1 = "Excelente produto chegou antes do prazo indico e recomendo produto bom pois já testei e foi mais que aprovado" 
frase2 = "SUPER RECOMENDO, PREÇO, QUALIDADE #BRASTEMP, EFICIÊNCIA NA ENTREGA, E FACILIDADE DE PAGAMENTO. MUITO BOM!!!"
frase3 = "A tampa do fogão veio com problemas com o pino de encaixe solto e precisa de reparos"
frase4 = "Fogão ótimo!"

## Distância de Jaccard

<b> Atividade </b>

1) Faça um método que calcule a similaridade de Jaccard e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: lembrando que você precisa aplicar um pre-processamento nessas frases antes de aplicar o método.
Faça:

- Lower
- Remoção StopWords
- Remoção Pontuação
- Tokenização

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/douglas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
def pre_processamento_texto(corpus):

    #tokenizacao
    corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)
    #capitalizacao
    corpus_alt = [t.lower() for t in corpus_alt]
    #remover stopwords
    stopwords_ = stopwords.words("portuguese")
    corpus_alt = [t for t in corpus_alt if t not in stopwords_]
    #remover pontuacoes
    corpus_alt = [t for t in corpus_alt if t not in string.punctuation]

    return corpus_alt

In [13]:
frase1_pre_processada = pre_processamento_texto(frase1)
frase2_pre_processada = pre_processamento_texto(frase2)
frase3_pre_processada = pre_processamento_texto(frase3)
frase4_pre_processada = pre_processamento_texto(frase4)

In [14]:
frase1_pre_processada

['excelente',
 'produto',
 'chegou',
 'antes',
 'prazo',
 'indico',
 'recomendo',
 'produto',
 'bom',
 'pois',
 'testei',
 'aprovado']

In [15]:
def jaccard_similarity(f1, f2):
    f1 = set(f1)
    f2 = set(f2)
    
    intersecao = f1.intersection(f2)
    uniao = f1.union(f2)
    
    return len(intersecao) / len(uniao)

In [16]:
print(jaccard_similarity(frase1_pre_processada, frase2_pre_processada))
print(jaccard_similarity(frase1_pre_processada, frase3_pre_processada))
print(jaccard_similarity(frase2_pre_processada, frase3_pre_processada))
print(jaccard_similarity(frase1_pre_processada, frase4_pre_processada))

0.10526315789473684
0.0
0.0
0.0


<b> Atividade </b>

2) Qual par de frase teve maior simlaridade? E qual teve menor? Este resultado faz sentido? Explique.

O par frase1 e frase2, pois as duas frases possuem algumas palavras em comum, logo a inerseção entre elas é diferente de zero.

## Distância de Cosseno

Aqui iremos calcular a distância do cosseno utilizando duas formas, que aprendemos na aula passada, para representar o texto.

- Bag of Words (BOW) 
- Embedding

Observação:

Existem duas formas de trabalhar com o cosseno:

<b> Distância </b>: quanto menor mais perto estão as frases.
<b> Similaridade </b>: quanto maior mais perto estão as frases.

### BOW - Distância do cosseno

<b> Atividade </b>

3) Calcule a distância do cosseno utilizando a representação CountVectorizer e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: no CountVectorizer utilizem as frases já pre-processadas da atividade anterior. Mas para aplicá-las no fit_transform, cada frase deve ser um string (sem estar tokenizada) dentro de uma lista.

```python
#exemplo
distance.cosine(frase1, frase2)
```

In [17]:
bow = CountVectorizer()

In [18]:
frases = [' '.join(frase1_pre_processada), ' '.join(frase2_pre_processada),
          ' '.join(frase3_pre_processada), ' '.join(frase4_pre_processada)]

In [19]:
vector_bow = bow.fit_transform(frases)

In [20]:
vector_bow

<4x29 sparse matrix of type '<class 'numpy.int64'>'
	with 32 stored elements in Compressed Sparse Row format>

In [21]:
vector_bow.todense()

matrix([[1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 2, 0,
         1, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
         1, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,
         0, 1, 1, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 1]])

In [22]:
frase1_bow = vector_bow.todense()[0]
frase2_bow = vector_bow.todense()[1]
frase3_bow = vector_bow.todense()[2]
frase4_bow = vector_bow.todense()[3]

In [23]:
frase1_bow

matrix([[1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 2, 0,
         1, 0, 0, 0, 0, 1, 0, 0]])

In [24]:
print(distance.cosine(frase1_bow, frase2_bow))
print(distance.cosine(frase1_bow, frase3_bow))
print(distance.cosine(frase2_bow, frase3_bow))
print(distance.cosine(frase1_bow, frase4_bow))

0.8309691490542968
1.0
1.0
1.0


<b> Atividade </b>


4) Qual par de frase teve maior distância? E qual teve menor? Este resultado faz sentido? Explique.

Os pares de frases frase1 e frase3, frase2 e frase3, e frase 1 e frase4 tiveram as maiores distâncias. Isso porque estas frases não têm nenhuma palavra em comum.

O par com menor distâcia foi o frase1 e frase2, pois possuem lagumas palavras em comum.

### Embedding - Distância do cosseno

Para calcular o embedding de cada uma das frases, utilize o modelo carregado inicialmente. 

Cada palavra tem um vetor, para formar o embedding da frase tire a média de todos os vetores.

Utilize as frases já pre-processadas

<b> Atividade </b> 

5) Calcule a distância do cosseno utilizando a representação Embedding e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4


In [25]:
word_vectors

In [96]:
def calcula_embedding(frase):
    return np.mean(np.array([word_vectors[palavra] for palavra in frase if palavra in word_vectors.vocab]))

In [27]:
frase1_vector_embedding = calcula_embedding(frase1_pre_processada)
frase2_vector_embedding = calcula_embedding(frase2_pre_processada)
frase3_vector_embedding = calcula_embedding(frase3_pre_processada)
frase4_vector_embedding = calcula_embedding(frase4_pre_processada)

In [28]:
frase1_vector_embedding

array([ 0.3227    ,  0.34005833, -0.12894166, -0.10406666, -0.4845917 ,
       -0.29425833,  0.12635   ,  0.49035   , -0.2429    ,  0.522125  ,
        0.20710002,  0.09579167, -0.485775  ,  0.06400833,  0.05996667,
        0.0134    ,  0.36394164, -0.3865333 , -0.19614166,  0.04160834,
        0.129625  ,  0.133325  , -0.13795833, -0.16118333, -0.12135834,
        0.07021666, -0.34681666, -0.41391668, -0.09325833,  0.26530832,
       -0.09575   , -0.03753333,  0.25401667,  0.310675  ,  0.01769167,
       -0.173275  ,  0.04813334,  0.27826664,  0.257025  , -0.25668335,
        0.30625832,  0.06915832, -0.44500837, -0.09507499,  0.01294167,
       -0.047425  , -0.19609167, -0.0149    , -0.67545   ,  0.26777497,
        0.36566666,  0.06413333, -0.06512501, -0.02040833,  0.37386668,
       -0.24394165, -0.18075836, -0.19190834, -0.04534167,  0.16798331,
        0.5301833 , -0.16669166, -0.23428333,  0.25604165, -0.03947499,
       -0.05850834, -0.33455002,  0.160325  ,  0.03743334, -0.01

In [29]:
print(distance.cosine(frase1_vector_embedding, frase2_vector_embedding))
print(distance.cosine(frase1_vector_embedding, frase3_vector_embedding))
print(distance.cosine(frase2_vector_embedding, frase3_vector_embedding))
print(distance.cosine(frase1_vector_embedding, frase4_vector_embedding))

0.16096973419189453
0.2234203815460205
0.2526938319206238
0.29043054580688477


In [30]:
print(frase1)
print(frase2)
print(frase3)
print(frase4)

Excelente produto chegou antes do prazo indico e recomendo produto bom pois já testei e foi mais que aprovado
SUPER RECOMENDO, PREÇO, QUALIDADE #BRASTEMP, EFICIÊNCIA NA ENTREGA, E FACILIDADE DE PAGAMENTO. MUITO BOM!!!
A tampa do fogão veio com problemas com o pino de encaixe solto e precisa de reparos
Fogão ótimo!


<b>Atividade </b>

6) Qual par de frase teve maior distância? E qual teve menor? Este resultado faz sentido? Explique.

O par de frases com maior distância foi o frase1 e frase4. A que teve menor distância foi o par frase1 e frase2.

O resultado faz sentido para as frases com menor distânica, pois ambas falam positivamente de algum produto.
Já o resulatdo da frase1 com a frase4 deveria ter tido uma distância menor, pois as duas frases também falam positivamente de algum produto.

Como estamos recuperando a médias dos embeddings, pode ser que o vetor de alguma(s) palavra(s) das duas frases esteja "pesando" mais em relação aos outros e distanciando as médias dos embeddings fazendo a distâcia entre as frases ficar maior.

## WMD

O WMD já está incorporado ao Word2Vec

<b> Atividade </b>

7) Calcule a distância WMD e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: use a variável já tokenizada.

In [31]:
print(word_vectors.wmdistance(frase1_pre_processada, frase2_pre_processada))
print(word_vectors.wmdistance(frase1_pre_processada, frase3_pre_processada))
print(word_vectors.wmdistance(frase2_pre_processada, frase3_pre_processada))
print(word_vectors.wmdistance(frase1_pre_processada, frase4_pre_processada))

3.0725697404938526
3.759219170455064
3.9048993635316385
3.851623338336251


<b> Atividade </b>

8) Qual par de frase teve maior distância? E qual teve menor? Este resultado faz sentido? Explique.

O par de frases com a maior distância foi frase2 e frase3. O par com menor distância foi frase1 e frase2.

O resultado faz sentido, pois as frases frase2 e frase3 expressam opniões diverntes em relação a um determinado produto: a frase2 fala positivamente sobre um produto, enquanto a frase3 fala negativamente.

Para o par frase1 e frase2 também faz sentido, pois ambas as frases falam positivamente de um determinado produto.

# Classificação de Documentos

A clssificação de documentos é muito útil em vários aspectos. Um dos tipos de classificação de texto é a análise de sentimentos.

A fim de ilustrar a classificação de documentos iremos criar um modelo para classificar uma frase como positiva ou negativa.

## Dataset

<b> Atividade </b>

9) Carregue o dataset com o pandas e depois dê o head no dataframe.


Link download: https://drive.google.com/open?id=15azJWdEEPGsXQGiDmEOseTBJcquWvBQc

<b> Este dataset é sobre revisões de filmes do IMDB. </b>

In [32]:
df = pd.read_csv("imdb-reviews-pt-br.csv")

In [33]:
df.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


## Representação dos dados

O sentimento positivo e negativo iremos binarizar cada um deles. Seja 1 positivo e 0 negativo.

Iremos representar o texto de duas formas:

- Bag of Words (BOW)
- Embedding

Depois iremos comparar o resultado de cada um deles.

### Representação Target

<b> Atividade </b>

10) Faça a representação dos sentimentos. 1 positivo; 0 negativo

In [34]:
target = df["sentiment"].replace(["neg", "pos"], [0, 1])

In [35]:
target

0        0
1        0
2        0
3        0
4        0
        ..
49454    1
49455    1
49456    1
49457    1
49458    1
Name: sentiment, Length: 49459, dtype: int64

### Bag of Words (BOW)

<b> Atividade </b>

11) Aplique o pré-processamento listado abaixo na coluna ``text_pt`` (crie uma nova coluna ```text_pt_sem_stopwords``` no dataframe para armazenar este dado processado):

- Remova as stopwords do texto
- Remova as pontuções
- Mantenha o texto sem tokenização, ou seja uma string

<b> Dica: </b> use o ```progress_apply``` para exibir a barra de progresso:

```python
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
df["colunas"].progress_apply(lambda x: preprocessamento(x))
```

In [50]:
def pre_processamento_string(corpus):

    #tokenizacao
    corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)
    #capitalizacao
    corpus_alt = [t.lower() for t in corpus_alt]
    #remover stopwords
    stopwords_ = stopwords.words("portuguese")
    corpus_alt = [t for t in corpus_alt if t not in stopwords_]
    #remover pontuacoes
    corpus_alt = [t for t in corpus_alt if t not in string.punctuation]
    
    corpus_alt = ' '.join(corpus_alt)
    return corpus_alt

In [82]:
df["text_pt_sem_stopwords"] = df["text_pt"].progress_apply(lambda x : pre_processamento_string(x))

<b> Atividade </b>

12) Aplique a representação do texto processado anteriormente com CountVectorizer.

In [38]:
bow = CountVectorizer()

In [52]:
frases = df["text_pt_sem_stopwords"]

In [53]:
frases.values

array(['vez sr costner arrumou filme tempo necessário além terríveis seqüências resgate mar quais poucas simplesmente importei nenhum personagens maioria fantasmas armário personagem costers realizado logo início esquecido tarde importava personagem deveríamos importar arrogante superconfiante ashton kutcher problema sai garoto pensa melhor qualquer outra pessoa redor mostra sinais armário desordenado único obstáculo parece estar vencendo costner finalmente bem além meio caminho costner conta sobre fantasmas kutchers informados kutcher levado ser melhor pressentimentos presságios anteriores nenhuma mágica aqui tudo podia fazer desligar hora',
       'exemplo motivo maioria filmes ação mesmos genérico chato nada valha pena assistir aqui completo desperdício talentos ice t cubo gelo mal aproveitados cada comprovando capazes atuar agir bem incomode vá ver new jack city ricochet assistir new york undercover ice t boyz hood higher learning friday ice cube ver negócio real ice ts horrivelmen

In [54]:
vector_bow = bow.fit_transform(frases)

In [55]:
vector_bow

<49459x129447 sparse matrix of type '<class 'numpy.int64'>'
	with 5072221 stored elements in Compressed Sparse Row format>

### Embedding

<b> Atividade </b>

13) Aplique o pré-processamento listado abaixo na coluna ``text_pt`` (crie uma nova coluna ```text_pt_sem_stopwords_token``` no dataframe para armazenar este dado processado):

- Aplique lower
- Remova as stopwords do texto
- Remova as pontuções
- Mantenha o texto com tokenização

<b> Dica: </b> use o ```progress_apply``` para exibir a barra de progresso:

```python
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
df["colunas"].progress_apply(lambda x: preprocessamento(x))
```

In [83]:
def pre_processamento_token(corpus):

    #tokenizacao
    corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)
    #capitalizacao
    corpus_alt = [t.lower() for t in corpus_alt]
    #remover stopwords
    stopwords_ = stopwords.words("portuguese")
    corpus_alt = [t for t in corpus_alt if t not in stopwords_]
    #remover pontuacoes
    corpus_alt = [t for t in corpus_alt if t not in string.punctuation]

    return corpus_alt

In [84]:
df["text_pt_sem_stopwords_token"] = df["text_pt"].progress_apply(lambda x : pre_processamento_token(x))


<b> Atividade </b>

14) Aplique a representação do texto com Embeddings. Cada palavra tem um embedding, o embedding da frase é a média de todos embeddings.

In [59]:
vetor_embedding = df["text_pt_sem_stopwords_token"].progress_apply(lambda x : calcula_embedding(x))

## Treinamento

### CountVectorizer


<b> Atividade </b>

15) Faça a divisão dados dados em treino e teste como no exemplo abaixo:

```python
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X_bag, target,random_state=123)
```

In [85]:
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(vector_bow, target,random_state=123)

In [86]:
X_train_bow.shape

(37094, 129447)

In [87]:
y_train_bow.shape

(37094,)

In [88]:
X_test_bow.shape

(12365, 129447)

In [89]:
y_test_bow.shape

(12365,)


<b> Atividade </b>

16) Treine com uma regressão logística

In [90]:
modelo_bow = LogisticRegression()
modelo_bow.fit(X_train_bow, y_train_bow)

LogisticRegression()


<b> Atividade </b>

17) Calcule as métricas de resultado utilizando método abaixo:

```python
print(classification_report(y_test_bow, y_pred))
```

In [91]:
preds = modelo_bow.predict(X_test_bow)

In [92]:
print(classification_report(y_test_bow, preds))

              precision    recall  f1-score   support

           0       0.89      0.87      0.88      6112
           1       0.87      0.89      0.88      6253

    accuracy                           0.88     12365
   macro avg       0.88      0.88      0.88     12365
weighted avg       0.88      0.88      0.88     12365



### Embedding


<b> Atividade </b>

18) Faça a divisão dados dados em treino e teste como no exemplo abaixo:

Verifique o shape do X treino e X teste. Caso eles estejam com apenas uma dimensão, você precisa tranformá-los para duas dimensões, caso contrário ocorrerá erro no treinamento.

In [105]:
X_train_emb, X_test_emb, y_train_emb, y_test_emb = train_test_split(vetor_embedding, target,random_state=123)

In [106]:
X_train_emb = pd.DataFrame([x for x in X_train_emb])

In [107]:
X_train_emb.shape

(37094, 100)

In [108]:
vetor_embedding[1000].shape

(100,)


<b> Atividade </b>

19) Treine com uma regressão logística

In [109]:
modelo_emb = LogisticRegression()
modelo_emb.fit(X_train_emb, y_train_emb)

LogisticRegression()


<b> Atividade </b>

20) Calcule as métricas de resultado utilizando método abaixo:

```python
print(classification_report(y_test_bow, y_pred))
```

#### Calcule as métricas

In [111]:
X_test_emb = pd.DataFrame([x for x in X_test_emb])

In [112]:
preds = modelo_emb.predict(X_test_emb)

In [114]:
print(classification_report(y_test_emb, preds))

              precision    recall  f1-score   support

           0       0.76      0.77      0.77      6112
           1       0.78      0.77      0.77      6253

    accuracy                           0.77     12365
   macro avg       0.77      0.77      0.77     12365
weighted avg       0.77      0.77      0.77     12365




<b> Atividade </b>

21) Compare os resultados obtidos com o BagOfWords e com o Embedding. Explique os possíveis motivos desta diferença.

# Análise de sentimentos

O modelo que criamos anteriormente é para ilustrar como podemos realizar classificação de documentos.
Quando a tarefa é sobre análise de sentimentos, temos duas opções: treinar nosso próprio modelo, como feito anteriormente ou utilizar uma das inúmeras ferramentas prontas.

Vamos testar as seguintes ferramentas:

- Vader
- Textblob
- Affin

Nesta atividade iremos utilizar as duas variáveis abaixo:

In [115]:
texto_neg = df.loc[0, "text_en"]
texto_pos = df.loc[49431, "text_en"]

## Vader

<b> Apenas Inglês </b>

O VADER (Valence Aware Dictionary e sEntiment Reasoner) é uma ferramenta de análise de sentimentos baseada em regras e léxico, especificamente identifica os sentimentos expressos nas mídias sociais.

- positive sentiment: compound score >= 0.05
- neutral sentiment: (compound score > -0.05) e (compound score < 0.05)
- negative sentiment: compound score <= -0.05

Mais informações: https://github.com/cjhutto/vaderSentiment


<b> Atividade </b>

22) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores(texto)
```

In [116]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [120]:
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores(texto_neg)

{'neg': 0.126, 'neu': 0.76, 'pos': 0.114, 'compound': 0.3958}

In [121]:
analyzer.polarity_scores(texto_pos)

{'neg': 0.084, 'neu': 0.737, 'pos': 0.179, 'compound': 0.9969}

## TextBlob

<b> Apenas inglês </b>

https://www.presentslide.in/2019/08/sentiment-analysis-textblob-library.html

<b> Atividade </b>
 
23) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
sentence=TextBlob(texto)
sentence.sentiment
```

In [122]:
from textblob import TextBlob

In [124]:
sentence=TextBlob(texto_neg)
sentence.sentiment

Sentiment(polarity=0.06385964912280702, subjectivity=0.5629824561403508)

In [125]:
sentence=TextBlob(texto_pos)
sentence.sentiment

Sentiment(polarity=0.190819118692253, subjectivity=0.6026226012793177)

## Afinn

- Valor maior que 0 indica sentimento positivo
- Valor menor que 0 indica sentimento negativo

<b> Atividade </b>

24) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
afinn = Afinn()
afinn.score(texto)
```

In [126]:
from afinn import Afinn

In [128]:
afinn = Afinn()
afinn.score(texto_neg)

8.0

In [129]:
afinn.score(texto_pos)

55.0

<b> Atividade </b>

25) Para você, qual ferramenta teve melhor comportamento?

Para mim a Vader teve um comportamento melhor, pois apresenta os sentimentos clasificados por positivos, negativos, neutros e mistos.

# Dica:
## Quando for trabalhar com um dataset em inglês, a biblioteca Spacy facilita!

In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
import spacy
import pandas as pd

O scpay forne um pacote que já tem série de modelos já treinados em NLP. Inclusive para os embeddings em inglês.

Para mais informações vá em:

https://spacy.io/models/en#en_core_web_md



Com o método abaixo carregamos um dos modelos do spacy:

```python
nlp = spacy.load('en_core_web_md')
```

Para aplicar o modelo, basta passar o texto para o modelo carregado anteriormente:

```python
doc = nlp("This is some text that I am processing with Spacy")
```

Carregue o modelo e imprima doc

In [ ]:
nlp = spacy.load('en_core_web_md')

In [ ]:
doc = nlp("This is some text that I am processing with Spacy")

In [ ]:
doc

Ao aplicar o modelo carregado a variável <b> doc </b> já possui os embeddings de cada uma das palavras e o embedding da frase, que é a média de todos vetores de todas palavras.

```python
#vetor da primeira palavra
doc[0].vector
#vetor agregado pela média - embedding do documento
doc.vector
```
O código abaixo mostra que a média de uma posição em específico dos embeddings de todas as palavras e a posição do embedding do documento possuem o mesmo valor.

In [ ]:
def calcula_media_posicao(x):
    soma = 0
    vector = []
    for i in range(0,len(doc)):
        vector.append(doc[i].vector)    
    
    for v in vector:
        soma += v[x]
    return soma/len(doc)

In [ ]:
round(calcula_media_posicao(10),6)

In [ ]:
round(doc.vector[10], 6)